In [1]:
import numpy as np
import torch as th
import torch.autograd as ag
import torch.nn.functional as F
import torch.nn as nn
import csv
th.manual_seed(1) # set the seed 

#import imdb


In [2]:
fname = "/home/nafaa/Documents/Projet_TER/Sentiment Analysis Dataset.csv"
file = open(fname, "r")
labels= [] 
train = []

   
reader = csv.reader(file)

for row in reader:
       
        train.append(row[3].lstrip().strip())
        labels.append(row[1])
#supprimer la colonne sentiment et sentiment text        
train.remove('SentimentText')  
labels.remove('Sentiment')   


In [3]:
print(len(labels))
print(len(train))

1578614
1578614


In [4]:
# 500 0000 données pour le test 
train_test=train[-500000:]
labels_test=labels[-500000:]
# 1 million données pour l'apprentissage 
train=train[:1000000]
labels=labels[:1000000]


In [5]:
print(len(labels_test))
print(len(train_test))
print(len(labels))
print(len(train))

500000
500000
1000000
1000000


In [ ]:
#mettre les donnes dans un array 
train=np.asarray(train)
labels=np.asarray(labels)



In [8]:
#fonction pour séparer la phrase et avoir des mots 
def phrase_to_mot(x):
    word=""
    word_list=[]
    b=True
    for i in range(len(x)): # à l'aid de la méthode isalpha on vérifie si c'est alphabet 
        if x[i].isalpha() or x[i]=="@" or x[i]=="&":
            if b:
                word=word+x[i]
                if i+1==len(x):
                    word_list.append(word)
            else:
                b=True
                word= word+x[i]

        else:
            b=False
            if word!="":
                word_list.append(word)
            word=""
        if word=="http" or word=="www":
            word_list.append(x)
            break
   
    return word_list



In [9]:
# dictionnaire contient tout les mots 
dic={}
dic_inverse={}
couts=[]
k=0
for sentense in train:
    word_list=phrase_to_mot(sentense)
    for word in word_list:
        if word not in dic:
            dic[word]=k
            k+=1
            dic_inverse[k]=word
            couts.append(1)
        else:
            index=dic[word]
            couts[index]+=1

In [23]:
# un nouveau dictionnaire qui contient les mots qui apparait plus de 20 fois et moins de 10000 fois
nouveau_dic={}
nouveau_dic_inverse={}
k=1
for i in range(len(couts)):
    if (couts[i]>20):
        if (couts[i]<10000):
            word=dic_inverse[i]
            nouveau_dic[word]=k
            nouveau_dic_inverse[k]=word
            k+=1
            

In [24]:
# fonction qui associé un nombre(la clé du mot dans le dictionnaire ) à chaque mot , si il n'existe pas dans le dictionnaire on le met 0
def compte(sentense):
    word_list=phrase_to_mot(sentense)
    l=[]
    for i in range(len(word_list)):
        if word_list[i] in nouveau_dic:
            k=nouveau_dic[word_list[i]]
            if k not in l:
                l.append(k)
        else:
            l.append(0)
    return l

In [25]:
print(compte(train[15]))
print(train[15])

[71, 72, 1, 5, 73, 27, 74, 75, 0]
&lt;-------- This is the way i feel right now...


In [26]:
#construire nouveau train et test , contient une list des tableaux convertie en entier  
nouveau_train=[]
nouveau_test=[]
for tweet in train:
    nouveau_train.append(compte(tweet))
train1=np.asarray(nouveau_train)

for sentense in train_test:
    nouveau_test.append(compte(sentense))
test1=np.asarray(nouveau_test)

In [27]:
#convertir les labels en entier 
labels=np.asarray([int(y) for y in labels])
labels_test=np.asarray([int (y ) for y in labels_test])

In [28]:
#une fonction qui calcule le nombre des sentiments positifs dans les labels
def sentiment_positif(labl):
    k=0

    for i in labl:
            if i == 1:
                k= k+1

    return k

In [29]:
#une fonction qui calcule le nombre des sentiments négatifs dans les labels
def sentiment_negatif(labl):
    k=0

    for i in labl:
            if i != 1:
                k= k+1

    return k

In [30]:

train_proba=[] # train_proba[0] contient la probabilité des sentiments négatifs, train_proba[1] positifs
test_proba=[] # test_proba[0] contient la probabilité des sentiments négatifs, test_proba[1] positifs

#les sentiments positifs pour les données d'apprentissage
train_sentiment_1=sentiment_positif(labels)
#les sentiments negatifs pour les données d'apprentissage
train_sentiment_0=sentiment_negatif(labels)
#probabilité des sentiment négatifs 
train_proba.append(train_sentiment_0/float(len(labels)))
#probabilité des sentiment positifs 
train_proba.append(train_sentiment_1/float(len(labels)))

#les sentiments positifs pour les données de test
test_sentiment_1=sentiment_positif(labels_test)
#les sentiments negatifs pour les données de test
test_sentiment_0=sentiment_negatif(labels_test)
#probabilité des sentiment négatifs pour les données de test
test_proba.append(test_sentiment_0/float(len(labels_test)))
#probabilité des sentiment positifs pour les données de test
test_proba.append(test_sentiment_1/float(len(labels_test)))

In [32]:
#une fonction qui calcule combien de fois les mots apparaissent dans la phrase négative ou positive
def calcul_apparence_mot(data_set,label):
    positif=[0 for i in range(len(nouveau_dic)+1)] # un tableau pour sentiments positif meme taille que le dictionnaire +1
    negatif=[0 for i in range(len(nouveau_dic)+1)]
    for i in range(len(data_set)):
        sentence=data_set[i]
        for index in sentence:
            if label[i]==1:
                positif[index]+=1
            else:
                negatif[index]+=1
    return negatif,positif

In [33]:
neg,pos =calcul_apparence_mot(train1,labels) # mettre les résultats dans l1 et l2 
train_apparence=[]#train_apparence[0] stock combien de fois le mot est dans la phrase négative train_apparence[1] positif
train_apparence.append(neg)
train_apparence.append(pos)

In [34]:
def proba_juge(train_appa,nbr_sent_pos,nbr_sent_neg):
    p0=[]
    p1=[]
    for i in range(len(train_appa[1])):
        p1.append((train_appa[1][i]+1)/float((nbr_sent_pos+10000)))
        p0.append((train_appa[0][i]+1)/float((nbr_sent_neg+10000)))
    return p0,p1

In [37]:
train_p =[]
p0,p1=proba_juge(train_apparence,train_sentiment_1,train_sentiment_0)
train_p.append(p0)
train_p.append(p1)

In [38]:
def predict_class(test):
    standard_test=test
    standard_test=np.asarray(standard_test)
    p1=1
    p0=1
    label=[]
    for tweet in standard_test:
        for index in tweet:
            p1*=train_p[1][index]
            p0*=train_p[0][index]
        p1*=train_proba[1]
        p0*=train_proba[0]
        if p1>p0:
            label.append(1)
        else:
            label.append(0)
        p1=1
        p0=1
    return label

        

In [ ]:
Test=np.asarray(test1)
label=predict_class(Test)

label=np.asarray(label)
mis=0
for i in range(len(label)):
    if labels_test[i]!=label[i]:
        mis+=1
print ("perte: ",mis/float(len(label)))
print ("réussite: ",(len(label)-mis)/float(len(label)))

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True, max_features = 10000)
ctrain= cv.fit_transform(train)
ctest= cv.transform(train_test)


In [41]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB(alpha=0.5)
print (type(labels_test[0]))

<class 'numpy.int64'>


In [42]:
bnb.fit(ctrain,labels)

BernoulliNB(alpha=0.5, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
bnb.score(ctest,labels_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=False, max_features = 5000)
ctrain= cv.fit_transform(train)
ctest= cv.transform(train_test)


from sklearn.naive_bayes import MultinomialNB
bnb = MultinomialNB(alpha=0.5)
bnb.fit(ctrain,labels)
bnb.score(ctest,labels_test)